# Spatial Joining with fiona and shapely

## Notes 

- Joins longitude and latitude points from a pandas dataframe to shape files polygons (and properties).
- Uses ERSI postcode and lga shape files from the ABS website.
- Includes postcode and local government area options.
- Accuracy can be traded off for speed using the tolerance_setting parameter.


## Importing modules

In [1]:
import fiona
import pandas
from shapely.geometry import Point, asShape
import datetime as dt
from shapely import speedups
speedups.enable()

## Set option for lga or postcode
Note: Shapefiles contain polygon boundaries from 2016, take caution when joining data preceding this time

In [2]:
lga = True
postcode = False

In [3]:
#Default to LGA if both LGA and Postcode are selected.

if lga == False and postcode == False:
    lga = True
    
if lga == True and postcode == True:
    postcode = False

# Conditionally set shape file paths.

if postcode == True:
    shapefile_path = "/Users/danielcorcoran/Desktop/Shared/POA_16/POA_2016_AUST.shp"
    property_code_name = "POA_CODE16"
elif lga == True:
    shapefile_path = "/Users/danielcorcoran/Desktop/Shared/LGA_ABS_16/LGA_2016_AUST.shp"
    property_code_name = "LGA_CODE16" 

## Set path for file containing coordinates

In [4]:
#Set path for coordinates data, to be read in.

coord_path = "/users/danielcorcoran/Desktop/GNAF VIC MAY18/OUTPUT/GNAF_MERGED_SUBSET.csv"

## Import data

In [5]:
#Read in coordinate data as pandas dataframe

coord_data = pandas.read_csv(coord_path)

## Set column names containing longitude and latitude values 
Check the headers here and set the longitude and latitude column names below

In [6]:
coord_data.head()

,ROW_INDEX,ADDRESS_DETAIL_PID,POSTCODE,ADDRESS_SITE_PID,STREET_LOCALITY_PID,GNAF_PROPERTY_PID,ADDRESS_DEFAULT_GEOCODE_PID,LONGITUDE,LATITUDE,STREET_NAME,STREET_TYPE_CODE
0,0,GAVIC421913470,3910,422057181,VIC2026862,221874894.0,1355201,145.201960,-38.153666,SOUTHGATEWAY,NaN
1,1,GAVIC425744379,3047,425832689,VIC2067171,420982251.0,9879848,144.926337,-37.682051,WARANGA,CRESCENT
2,2,GAVIC425745093,3555,425833403,VIC2067610,423926943.0,9885130,144.265766,-36.782149,WOODWARD,ROAD
3,3,GAVIC425745094,3555,425833404,VIC2067610,423926944.0,9881933,144.266020,-36.782179,WOODWARD,ROAD
4,4,GAVIC425756883,3150,425845193,VIC2046955,424358119.0,9894125,145.157176,-37.877506,ROSE,AVENUE


In [7]:
latitude_column_name = "LATITUDE"
longitude_column_name = "LONGITUDE"

## Iterate through fiona collection and store polygons, simplified polygons, properties and boundaries
Creating reference data in the form of lists

In [8]:
#Note: Code is BUILT TO FIT the ABS LGA/Postcode shapefiles released in 2016. 
#Take caution when using data from preceding 2016.

fiona_collection = fiona.open(shapefile_path)
fiona_collection_list = list(fiona_collection)

shapes_list = []
simple_shapes_list = []
shapes_properties_list = []
shapes_bounds_list = []

tolerance_setting = 0.0000005

for index in range(len(fiona_collection_list)):
    
    if lga == True and postcode == False:
        current_shape = fiona_collection_list[index]
        state_name = current_shape["properties"]["STE_NAME16"]
        properties = current_shape["properties"]

        if state_name == "Victoria" and current_shape["geometry"] is not None:
            shapes_list.append(asShape(current_shape["geometry"]))
            simple_shapes_list.append(asShape(current_shape["geometry"]).simplify(tolerance = tolerance_setting))
            shapes_properties_list.append(current_shape["properties"])
            shapes_bounds_list.append(asShape(current_shape["geometry"]).bounds)
            
    elif lga == False and postcode == True:    
        current_shape = fiona_collection_list[index]
        postal_state_code = list(current_shape["properties"]["POA_CODE16"])[0]
        properties = current_shape["properties"]

        if postal_state_code == "3" and current_shape["geometry"] is not None:
            shapes_list.append(asShape(current_shape["geometry"]))
            simple_shapes_list.append(asShape(current_shape["geometry"]).simplify(tolerance = tolerance_setting))
            shapes_properties_list.append(current_shape["properties"])
            shapes_bounds_list.append(asShape(current_shape["geometry"]).bounds)

## Check the size of each list
Checking how many polygons we have in total, there should be 698 postcode polygons, or 80 lga polygons

In [9]:
len(shapes_list)

80

In [10]:
len(shapes_bounds_list)

80

In [11]:
len(simple_shapes_list)

80

In [12]:
len(shapes_properties_list)

80





## Calculate the master bounds
master_bounds corresponds to maximum and minimum coordinates for Victoria. These coordinates are required to speed up algorithm time.

In [13]:
master_bounds = [shapes_bounds_list[0][0], shapes_bounds_list[0][1],
shapes_bounds_list[0][2],shapes_bounds_list[0][3]]

In [14]:
for shape_bounds_tuple in shapes_bounds_list:
    
    current_long_low = shape_bounds_tuple[0]
    current_long_high = shape_bounds_tuple[2]
    current_lat_low = shape_bounds_tuple[1]
    current_lat_high = shape_bounds_tuple[3]
    
    master_long_low = master_bounds[0]
    master_long_high = master_bounds[2]
    master_lat_low = master_bounds[1]
    master_lat_high = master_bounds[3]
    
    if current_long_low < master_long_low:
        master_bounds[0] = current_long_low
        print("Master bounds longitude (low) has been updated to",master_bounds[0])
    
    if current_long_high > master_long_high:
        master_bounds[2] = current_long_high
        print("Master bounds longitude (high) has been updated to",master_bounds[2])
        
    if current_lat_low < master_lat_low:
        master_bounds[1] = current_lat_low
        print("Master bounds latitude (low) has been updated to",master_bounds[1])
        
    if current_lat_high > master_lat_high:
        master_bounds[3] = current_lat_high
        print("Master bounds latitude (high) has been updated to",master_bounds[3])

Master bounds longitude (low) has been updated to 142.353480213
Master bounds latitude (low) has been updated to -37.836583308999934
Master bounds latitude (low) has been updated to -38.67879995399994
Master bounds latitude (high) has been updated to -36.25505162799993
Master bounds latitude (high) has been updated to -35.19282674999994
Master bounds latitude (low) has been updated to -38.85769295599994
Master bounds longitude (high) has been updated to 149.9762909950001
Master bounds longitude (low) has been updated to 140.965735138
Master bounds longitude (low) has been updated to 140.96168199600004
Master bounds latitude (high) has been updated to -33.98042558699996
Master bounds latitude (low) has been updated to -39.13674005499996
Master bounds latitude (low) has been updated to -39.159189527999956


In [15]:
#Check master bounds

master_bounds

[140.96168199600004,
 -39.159189527999956,
 149.9762909950001,
 -33.98042558699996]

## Dictionary to store results

In [16]:
results = {"data_row_index":[],
           "matched_code":[]}

## Process, output results

In [ ]:
#Reset counter, contains every coordinate match against a shapefile polygon
contains = 0

#Start timer
start = dt.datetime.now()

#Iterate through coordinate data set, upper level loop
for data_index in range(coord_data.shape[0]):
    
    print("Starting check for geopoint #",data_index)
    
    longitude = coord_data.loc[data_index, longitude_column_name]
    latitude = coord_data.loc[data_index, latitude_column_name]
    point = Point(longitude, latitude)
    
    #Check if point is within the maximum bounds for victoria
    if master_bounds[0] <= longitude <= master_bounds[2] and master_bounds[1] <= latitude <= master_bounds[3]:

        #If within Victorias bounds, start checking against each lga polygon
        for polygon_index in range(len(shapes_bounds_list)):

                #Check if point is within maximum bounds for LGA
                if shapes_bounds_list[polygon_index][0] <= longitude <= shapes_bounds_list[polygon_index][2] and shapes_bounds_list[polygon_index][1] <= latitude <= shapes_bounds_list[polygon_index][3]:

                    #If within LGA box boundary, check if the polygon contains point
                    #simple_shapes_list contains 'reduced' lga polygons
                    if shapes_list[polygon_index].contains(point):

                        print("\t>Coordinates at data index",data_index,"contains point")

                        results["data_row_index"].append(data_index)
                        results["matched_code"].append(shapes_properties_list[polygon_index][property_code_name])
                        contains = contains + 1
                        
                        break

    else:
        print("\t>Shape at data index",data_index,"is outside of Victoria's bounds.")

#Show points found and time taken to run
end = dt.datetime.now()
difference_time = end - start

timestamp = end.strftime("%y%m%d_%H%M")

print("\n","POINTS FOUND:",contains)
print("POINTS TOTAL:",coord_data.shape[0])
print("FINISHED IN:",difference_time)

result_dataframe = pandas.DataFrame(results)
result_dataframe.to_csv("sjoin_output_"+timestamp+".csv", index = False)
print("Spine data was exported successfully. Use this dataset to join the original data to shape files.")

Starting check for geopoint # 0
	>Coordinates at data index 0 contains point
Starting check for geopoint # 1
	>Coordinates at data index 1 contains point
Starting check for geopoint # 2
	>Coordinates at data index 2 contains point
Starting check for geopoint # 3
	>Coordinates at data index 3 contains point
Starting check for geopoint # 4
	>Coordinates at data index 4 contains point
Starting check for geopoint # 5
	>Coordinates at data index 5 contains point
Starting check for geopoint # 6
	>Coordinates at data index 6 contains point
Starting check for geopoint # 7
	>Coordinates at data index 7 contains point
Starting check for geopoint # 8
	>Coordinates at data index 8 contains point
Starting check for geopoint # 9
	>Coordinates at data index 9 contains point
Starting check for geopoint # 10
	>Coordinates at data index 10 contains point
Starting check for geopoint # 11
	>Coordinates at data index 11 contains point
Starting check for geopoint # 12
	>Coordinates at data index 12 contains

	>Coordinates at data index 105 contains point
Starting check for geopoint # 106
	>Coordinates at data index 106 contains point
Starting check for geopoint # 107
	>Coordinates at data index 107 contains point
Starting check for geopoint # 108
	>Coordinates at data index 108 contains point
Starting check for geopoint # 109
	>Coordinates at data index 109 contains point
Starting check for geopoint # 110
	>Coordinates at data index 110 contains point
Starting check for geopoint # 111
	>Coordinates at data index 111 contains point
Starting check for geopoint # 112
	>Coordinates at data index 112 contains point
Starting check for geopoint # 113
	>Coordinates at data index 113 contains point
Starting check for geopoint # 114
	>Coordinates at data index 114 contains point
Starting check for geopoint # 115
	>Coordinates at data index 115 contains point
Starting check for geopoint # 116
	>Coordinates at data index 116 contains point
Starting check for geopoint # 117
	>Coordinates at data index 

	>Coordinates at data index 210 contains point
Starting check for geopoint # 211
	>Coordinates at data index 211 contains point
Starting check for geopoint # 212
	>Coordinates at data index 212 contains point
Starting check for geopoint # 213
	>Coordinates at data index 213 contains point
Starting check for geopoint # 214
	>Coordinates at data index 214 contains point
Starting check for geopoint # 215
	>Coordinates at data index 215 contains point
Starting check for geopoint # 216
	>Coordinates at data index 216 contains point
Starting check for geopoint # 217
	>Coordinates at data index 217 contains point
Starting check for geopoint # 218
	>Coordinates at data index 218 contains point
Starting check for geopoint # 219
	>Coordinates at data index 219 contains point
Starting check for geopoint # 220
	>Coordinates at data index 220 contains point
Starting check for geopoint # 221
	>Coordinates at data index 221 contains point
Starting check for geopoint # 222
	>Coordinates at data index 

	>Coordinates at data index 321 contains point
Starting check for geopoint # 322
	>Coordinates at data index 322 contains point
Starting check for geopoint # 323
	>Coordinates at data index 323 contains point
Starting check for geopoint # 324
	>Coordinates at data index 324 contains point
Starting check for geopoint # 325
	>Coordinates at data index 325 contains point
Starting check for geopoint # 326
	>Coordinates at data index 326 contains point
Starting check for geopoint # 327
	>Coordinates at data index 327 contains point
Starting check for geopoint # 328
	>Coordinates at data index 328 contains point
Starting check for geopoint # 329
	>Coordinates at data index 329 contains point
Starting check for geopoint # 330
	>Coordinates at data index 330 contains point
Starting check for geopoint # 331
	>Coordinates at data index 331 contains point
Starting check for geopoint # 332
	>Coordinates at data index 332 contains point
Starting check for geopoint # 333
	>Coordinates at data index 

	>Coordinates at data index 429 contains point
Starting check for geopoint # 430
	>Coordinates at data index 430 contains point
Starting check for geopoint # 431
	>Coordinates at data index 431 contains point
Starting check for geopoint # 432
	>Coordinates at data index 432 contains point
Starting check for geopoint # 433
	>Coordinates at data index 433 contains point
Starting check for geopoint # 434
	>Coordinates at data index 434 contains point
Starting check for geopoint # 435
	>Coordinates at data index 435 contains point
Starting check for geopoint # 436
	>Coordinates at data index 436 contains point
Starting check for geopoint # 437
	>Coordinates at data index 437 contains point
Starting check for geopoint # 438
	>Coordinates at data index 438 contains point
Starting check for geopoint # 439
	>Coordinates at data index 439 contains point
Starting check for geopoint # 440
	>Coordinates at data index 440 contains point
Starting check for geopoint # 441
	>Coordinates at data index 

...

	>Coordinates at data index 3794 contains point
Starting check for geopoint # 3795
	>Coordinates at data index 3795 contains point
Starting check for geopoint # 3796
	>Coordinates at data index 3796 contains point
Starting check for geopoint # 3797
	>Coordinates at data index 3797 contains point
Starting check for geopoint # 3798
	>Coordinates at data index 3798 contains point
Starting check for geopoint # 3799
	>Coordinates at data index 3799 contains point
Starting check for geopoint # 3800
	>Coordinates at data index 3800 contains point
Starting check for geopoint # 3801
	>Coordinates at data index 3801 contains point
Starting check for geopoint # 3802
	>Coordinates at data index 3802 contains point
Starting check for geopoint # 3803
	>Coordinates at data index 3803 contains point
Starting check for geopoint # 3804
	>Coordinates at data index 3804 contains point
Starting check for geopoint # 3805
	>Coordinates at data index 3805 contains point
Starting check for geopoint # 3806
	>Co

	>Coordinates at data index 3895 contains point
Starting check for geopoint # 3896
	>Coordinates at data index 3896 contains point
Starting check for geopoint # 3897
	>Coordinates at data index 3897 contains point
Starting check for geopoint # 3898
	>Coordinates at data index 3898 contains point
Starting check for geopoint # 3899
	>Coordinates at data index 3899 contains point
Starting check for geopoint # 3900
	>Coordinates at data index 3900 contains point
Starting check for geopoint # 3901
	>Coordinates at data index 3901 contains point
Starting check for geopoint # 3902
	>Coordinates at data index 3902 contains point
Starting check for geopoint # 3903
	>Coordinates at data index 3903 contains point
Starting check for geopoint # 3904
	>Coordinates at data index 3904 contains point
Starting check for geopoint # 3905
	>Coordinates at data index 3905 contains point
Starting check for geopoint # 3906
	>Coordinates at data index 3906 contains point
Starting check for geopoint # 3907
	>Co

	>Coordinates at data index 3999 contains point
Starting check for geopoint # 4000
	>Coordinates at data index 4000 contains point
Starting check for geopoint # 4001
	>Coordinates at data index 4001 contains point
Starting check for geopoint # 4002
	>Coordinates at data index 4002 contains point
Starting check for geopoint # 4003
	>Coordinates at data index 4003 contains point
Starting check for geopoint # 4004
	>Coordinates at data index 4004 contains point
Starting check for geopoint # 4005
	>Coordinates at data index 4005 contains point
Starting check for geopoint # 4006
	>Coordinates at data index 4006 contains point
Starting check for geopoint # 4007
	>Coordinates at data index 4007 contains point
Starting check for geopoint # 4008
	>Coordinates at data index 4008 contains point
Starting check for geopoint # 4009
	>Coordinates at data index 4009 contains point
Starting check for geopoint # 4010
	>Coordinates at data index 4010 contains point
Starting check for geopoint # 4011
	>Co

	>Coordinates at data index 4104 contains point
Starting check for geopoint # 4105
	>Coordinates at data index 4105 contains point
Starting check for geopoint # 4106
	>Coordinates at data index 4106 contains point
Starting check for geopoint # 4107
	>Coordinates at data index 4107 contains point
Starting check for geopoint # 4108
	>Coordinates at data index 4108 contains point
Starting check for geopoint # 4109
	>Coordinates at data index 4109 contains point
Starting check for geopoint # 4110
	>Coordinates at data index 4110 contains point
Starting check for geopoint # 4111
	>Coordinates at data index 4111 contains point
Starting check for geopoint # 4112
	>Coordinates at data index 4112 contains point
Starting check for geopoint # 4113
	>Coordinates at data index 4113 contains point
Starting check for geopoint # 4114
	>Coordinates at data index 4114 contains point
Starting check for geopoint # 4115
	>Coordinates at data index 4115 contains point
Starting check for geopoint # 4116
	>Co

	>Coordinates at data index 4204 contains point
Starting check for geopoint # 4205
	>Coordinates at data index 4205 contains point
Starting check for geopoint # 4206
	>Coordinates at data index 4206 contains point
Starting check for geopoint # 4207
	>Coordinates at data index 4207 contains point
Starting check for geopoint # 4208
	>Coordinates at data index 4208 contains point
Starting check for geopoint # 4209
	>Coordinates at data index 4209 contains point
Starting check for geopoint # 4210
	>Coordinates at data index 4210 contains point
Starting check for geopoint # 4211
	>Coordinates at data index 4211 contains point
Starting check for geopoint # 4212
	>Coordinates at data index 4212 contains point
Starting check for geopoint # 4213
	>Coordinates at data index 4213 contains point
Starting check for geopoint # 4214
	>Coordinates at data index 4214 contains point
Starting check for geopoint # 4215
	>Coordinates at data index 4215 contains point
Starting check for geopoint # 4216
	>Co

## Export coordinate data with index label

Data should be exported with data_index label to join to output files data_index column

In [ ]:
coord_data.to_csv("GNAF_MERGED_SUBSET_V2.csv", index = False)

print("Coordinate data was exported successfully.")